In [1]:
import numpy as np
import pandas as pd

In [2]:
data_pd = pd.read_csv("nf_data_1_processed",header=0)
data_pd.head(2)

,Unnamed: 0,Cust_Id,Rating,Date,Movie_Id
0,0,712664,5.0,2004-02-01,3
1,1,1331154,4.0,2004-07-03,3


In [3]:
data_pd = data_pd.rename(columns={"Cust_Id": "UserId", "Movie_Id": "ItemId","Date":'date'})

In [4]:
data_pd = data_pd[['UserId','ItemId','date','Rating']]
data_pd.head(2)

,UserId,ItemId,date,Rating
0,712664,3,2004-02-01,5.0
1,1331154,3,2004-07-03,4.0


In [5]:
print('time span:',data_pd['date'].min(), data_pd['date'].max())

time span: 1999-11-11 2005-12-31


In [6]:
print(data_pd.shape)

(17337458, 4)


In [7]:
data_pd.groupby('Rating').agg({'ItemId':'count'})

,ItemId
Rating,
1.0,721356
2.0,1759281
3.0,5185650
4.0,5901368
5.0,3769803


In [8]:
print('unique ids:', data_pd['UserId'].unique().shape, data_pd['ItemId'].unique().shape)

unique ids: (143458,) (1350,)


In [9]:
data_pd['date']=pd.to_datetime(data_pd['date'])

## Split slots

In [12]:
import datetime

In [20]:
data_pd['timestamp'] = data_pd['date'].astype(int)

In [25]:
time_min = data_pd['timestamp'].min()
time_max = data_pd['timestamp'].max()
slot_gap = (time_max - time_min) /10

In [26]:
slot_gap

1.937088e+16

In [27]:
data=data_pd

In [30]:
time_min = data['timestamp'].min()
time_max = data['timestamp'].max()
slot_gap = (time_max - time_min) /10
data['time_slot'] = data["timestamp"].apply(lambda x: int(min(int((x-time_min))//slot_gap,9)))
data['time_slot'] = data[['time_slot']].astype(np.int)

/tmp/ipykernel_187/3955024126.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['time_slot'] = data[['time_slot']].astype(np.int)


In [32]:
data.groupby('time_slot').count()

,UserId,ItemId,date,Rating,timestamp
time_slot,,,,,
0,73772,73772,73772,73772,73772
1,120046,120046,120046,120046,120046
2,165978,165978,165978,165978,165978
3,327145,327145,327145,327145,327145
4,501930,501930,501930,501930,501930
5,818362,818362,818362,818362,818362
6,1854628,1854628,1854628,1854628,1854628
7,3397493,3397493,3397493,3397493,3397493
8,5183034,5183034,5183034,5183034,5183034


In [33]:
train_slots = [0 ,1, 2, 3, 4, 5, 6,7,8]
test_slots = [9]
data_train = data[data['time_slot'].isin(train_slots)]
data_test = data[data['time_slot'].isin(test_slots)]
print("train:",data_train.shape[0],'test:',data_test.shape[0])

train: 12442388 test: 4895070


In [34]:
user_in_train = data_train['UserId'].unique()
item_in_train = data_train['ItemId'].unique()

# filter new user/item in train
data_test = data_test[data_test['UserId'].isin(user_in_train)]
print("user not include in user_items_test:",data_test.shape)
data_test = data_test[data_test['ItemId'].isin(item_in_train)]
print("train:",data_train.shape[0],'not-new test:',data_test.shape[0])

# filter repeat
data_train = data_train.drop_duplicates(subset=['UserId','ItemId'],keep='first')
data_test = data_test.drop_duplicates(subset=['UserId','ItemId'],keep='first')
print("not repeat train:",data_train.shape[0],'not-repeat test:',data_test.shape[0])

user not include in user_items_test: (2791715, 6)
train: 12442388 not-new test: 2776038
not repeat train: 12442388 not-repeat test: 2776038


In [35]:
user = data_train['UserId'].unique()
item= data_train['ItemId'].unique()
user_to_id = dict(zip(list(user),list(np.arange(user.shape[0]))))
item_to_id = dict(zip(list(item),list(range(item.shape[0]))))
print("user num:",user.shape)
print("item num:", item.shape)
data_train['uid'] = data_train['UserId'].map(user_to_id)
data_train['iid'] = data_train['ItemId'].map(item_to_id)

data_test['uid'] = data_test['UserId'].map(user_to_id)
data_test['iid'] = data_test['ItemId'].map(item_to_id)

data_test.head(2)

user num: (122958,)
item num: (1343,)


,UserId,ItemId,date,Rating,timestamp,time_slot,uid,iid
2,2632461,3,2005-07-22,3.0,1121990400000000000,9,100494,0
15,1694958,3,2005-08-10,3.0,1123632000000000000,9,100525,0


In [37]:
data_train = data_train[['uid','iid','time_slot','Rating']]
data_test = data_test[['uid','iid','time_slot','Rating']]
#
columns = ['uid','iid','time_slot','click']
data_train.columns = columns
data_test.columns = columns
data_test.head(2)

,uid,iid,time_slot,click
2,100494,0,9,3.0
15,100525,0,9,3.0


## split validation set

In [39]:
test_unique_user = data_test['uid'].unique()
N_ = test_unique_user.shape[0]
np.random.seed(2020)
np.random.shuffle(test_unique_user)
split_idx  = int(N_*0.7)
test_real_user = test_unique_user[:split_idx]
valid_real_user = test_unique_user[split_idx:]
print("tot user in the last stage:",N_,"real test user:",test_real_user.shape[0],"real valid user:",valid_real_user.shape[0])

tot user in the last stage: 107898 real test user: 75528 real valid user: 32370


In [41]:
data_real_test = data_test[data_test['uid'].isin(test_real_user)]
data_real_valid = data_test[data_test['uid'].isin(valid_real_user)]
print("tot itr:",data_test.shape,"real test:",data_real_test.shape,"real valid:",data_real_valid.shape)

tot itr: (2776038, 4) real test: (1942172, 4) real valid: (833866, 4)


## Write txt files (Not run in future)

In [55]:
user_items_test 

,uid,iid,time_slot,click
0,0,0,6,5.0
1,0,556,5,4.0
2,0,555,8,5.0
3,0,1089,7,3.0
4,0,549,7,3.0
...,...,...,...,...
12442383,122953,1333,8,3.0
12442384,122954,1333,8,5.0
12442385,122955,1333,8,3.0
12442386,122956,1333,8,1.0


In [42]:
import os
path_folder = "./netflix_movie/"
if not os.path.exists(path_folder):
    os.mkdir(path_folder)
user_items_test = data_real_test.sort_values(by='uid',ignore_index=True)
print(user_items_test.head(2))
test_itr = user_items_test.values[:,0:2]
print(test_itr.shape)
with open('./netflix_movie/test_real.txt','w') as f:
    u_pre = test_itr[0,0]
    k = 0
    for x in test_itr:
        u = x[0]
        i = x[1]
        if u !=u_pre or k==0:
            u_pre = u
            if k>0:
                f.write('\n')
            f.write(str(u))
            k = 1
        f.write(' '+str(i))

   uid  iid  time_slot  click
0    0  271          9    4.0
1    0  783          9    4.0
(1942172, 2)


In [44]:

user_items_test = data_real_valid.sort_values(by='uid',ignore_index=True)
print(user_items_test.head(2))
test_itr = user_items_test.values[:,0:2]
print(test_itr.shape)
with open('./netflix_movie/valid_real.txt','w') as f:
    u_pre = test_itr[0,0]
    k = 0
    for x in test_itr:
        u = x[0]
        i = x[1]
        if u !=u_pre or k==0:
            u_pre = u
            if k>0:
                f.write('\n')
            f.write(str(u))
            k = 1
        f.write(' '+str(i))

   uid  iid  time_slot  click
0    1  441          9    4.0
1    1  900          9    3.0
(833866, 2)


In [46]:
user_items_test = data_train.sort_values(by='uid',ignore_index=True)
print(user_items_test.head(2))
test_itr = user_items_test.values[:,0:2]
print(test_itr.shape)
with open('./netflix_movie/train.txt','w') as f:
    u_pre = test_itr[0,0]
    k = 0
    for x in test_itr:
        u = x[0]
        i = x[1]
        if u !=u_pre or k==0:
            u_pre = u
            if k>0:
                f.write('\n')
            f.write(str(u))
            k = 1
        f.write(' '+str(i))

   uid  iid  time_slot  click
0    0    0          6    5.0
1    0  556          5    4.0
(12442388, 2)


In [47]:
data_train.to_csv("./netflix_movie/train_with_time.txt",index=False,header=False,sep=' ')

In [48]:
data_real_valid.to_csv("./netflix_movie/valid_with_time.txt",index=False,header=False,sep=' ')
data_real_test.to_csv("./netflix_movie/test_with_time.txt",index=False,header=False,sep=' ')

In [49]:
data_train['iid'].max()

1342

In [51]:
for slot_id in train_slots:
    slot_data = data_train[data_train['time_slot'].isin([slot_id])]
    slot_data  = slot_data.sort_values(by=['iid'],ignore_index=True)
    slot_data_np = slot_data[['iid','uid']].values[:,0:2]
    print(slot_data.head(2))
    print(slot_data[['iid','uid']].head(2))
    print(slot_data.shape)
    with open("./netflix_movie/t_"+str(slot_id)+".txt",'w') as f:
        i_pre = slot_data_np[0,0]
        k = 0
        for x in slot_data_np:
            i_ = x[0]
            u_ = x[1]
            if i_ != i_pre or k == 0:
                i_pre = i_
                if k>0:
                    f.write('\n')
                f.write(str(i_))
                k = 1
            f.write(" " + str(u_))

    uid  iid  time_slot  click
0  5428    2          0    2.0
1  6283    2          0    1.0
   iid   uid
0    2  5428
1    2  6283
(73772, 4)
    uid  iid  time_slot  click
0  5473    2          1    2.0
1  6297    2          1    3.0
   iid   uid
0    2  5473
1    2  6297
(120046, 4)
    uid  iid  time_slot  click
0  5430    2          2    3.0
1  6572    2          2    3.0
   iid   uid
0    2  5430
1    2  6572
(165978, 4)
    uid  iid  time_slot  click
0  5429    2          3    3.0
1  6597    2          3    5.0
   iid   uid
0    2  5429
1    2  6597
(327145, 4)
    uid  iid  time_slot  click
0  5443    2          4    3.0
1  6568    2          4    4.0
   iid   uid
0    2  5443
1    2  6568
(501930, 4)
   uid  iid  time_slot  click
0    5    0          5    3.0
1  834    0          5    3.0
   iid  uid
0    0    5
1    0  834
(818362, 4)
   uid  iid  time_slot  click
0    0    0          6    5.0
1  827    0          6    3.0
   iid  uid
0    0    0
1    0  827
(1854628, 4)
   u

In [53]:
slot_data = data_test
slot_data  = slot_data.sort_values(by=['iid'],ignore_index=True)
slot_data_np = slot_data[['iid','uid']].values[:,0:2]
print(slot_data.head(2))
print(slot_data[['iid','uid']].head(2))
print(slot_data.shape)
with open("./netflix_movie/t_"+str(9)+".txt",'w') as f:
    i_pre = slot_data_np[0,0]
    k = 0
    for x in slot_data_np:
        i_ = x[0]
        u_ = x[1]
        if i_ != i_pre or k == 0:
            i_pre = i_
            if k>0:
                f.write('\n')
            f.write(str(i_))
            k = 1
        f.write(" " + str(u_))

      uid  iid  time_slot  click
0  100494    0          9    3.0
1   60061    0          9    5.0
   iid     uid
0    0  100494
1    0   60061
(2776038, 4)
